#### Start

In [ ]:
# mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load token from file
import os
with open('/content/drive/MyDrive/hf_token.txt', 'r') as file:
    hf_token = file.read().strip()

### Text extraction from the pdfs - run if required

In [ ]:
!pip install PyPDF2

In [ ]:
import PyPDF2
import os
# Folder path containing multiple PDF files
folder_path = '/content/drive/MyDrive/data_argi_llm/kvk_pop/LAKSHADWEEP/lakshadweep/lakshadweep'

In [ ]:
# Function to read and extract text from a PDF file
def extract_text_from_pdf(pdf_file):
    text = ""
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

# Loop through each file in the folder and extract text if it's a PDF
for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(folder_path, filename)
        pdf_text = extract_text_from_pdf(pdf_path)

        # Save the extracted text to a text file with the same name as the PDF
        output_filename = filename.replace('.pdf', '.txt')
        output_path = os.path.join(folder_path, output_filename)
        with open(output_path, 'w') as output_file:
            output_file.write(pdf_text)
        print(f"Extracted text from {filename} and saved to {output_filename}")


### Creation of dataset using extracted text - if required

In [ ]:
!pip install -U bitsandbytes
!pip install accelerate
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes as bnb
import torch
import os

In [ ]:

# Load the model and tokenizer with 4-bit quantization
model_name = "NousResearch/Llama-2-7b-chat-hf"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Load the model with 4-bit precision using BitsAndBytesConfig
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='auto',  # Automatically map to available devices (GPU/CPU)
    token=token
)


In [ ]:
# Read the input text from text file
file_list = [f for f in os.listdir(folder_path)\
             if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('.txt')]
with open(os.path.join(folder_path, file_list[0]), 'r') as file:
    input_text = file.read()

# Split input text into smaller chunks (e.g., 2000 characters each)
chunk_size = 2000
text_chunks = [input_text[i:i + chunk_size] for i in range(0, len(input_text),\
                                                           chunk_size)]


In [ ]:
# Initialize a list to store formatted responses
formatted_responses = []

# Process each chunk and generate instruction-response pairs
for chunk in text_chunks:
    prompt = f"Extract all possible questions and answers from the following text:\n\n{chunk}\n\nFormat them as 'Q: <question>' and 'A: <answer>'."

    # Tokenize the prompt into pytorch tensors
    inputs = tokenizer(prompt, return_tensors="pt")

    # Calculate the length of the prompt in tokens
    prompt_length = inputs["input_ids"].shape[-1]

    # Move the input tensors to the same device (machine) as the model
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    # Generate outputs with increased max_new_tokens
    outputs = model.generate(**inputs, max_new_tokens=prompt_length+600, num_return_sequences=1)

    # Slice the generated output to remove the initial prompt tokens
    output = outputs[0][prompt_length:]

    # Decode and process the generated response
    response = tokenizer.decode(output, skip_special_tokens=True)

    # Extract only lines starting with "Q:" or "A:"
    questions_answers = []
    for line in response.split('\n'):
        if line.startswith("Q:") or line.startswith("A:"):
            questions_answers.append(line.strip())
    formatted_responses.extend(questions_answers)

In [ ]:
# Print the formatted responses
for item in formatted_responses:
    print(item)

In [ ]:
# Save the formatted responses to a file for training
with open(os.path.join(folder_path, 'data_ft.txt'), 'w') as f:
    for item in formatted_responses:
        f.write(f"{item}\n")